In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("../input/identify-snake-breed-hackerearth/dataset/train.csv")
test_df = pd.read_csv("../input/identify-snake-breed-hackerearth/dataset/test.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.breed.value_counts()

In [ ]:
train_df["breed"].unique()

In [ ]:
dls = ImageDataLoaders.from_df(train_df,path="../input/identify-snake-breed-hackerearth/dataset",
                               folder="train",
                               suff=".jpg",
                               label_col="breed",valid_pct=0.2,
                               item_tfms= Resize(224),
                               batch_tfms=aug_transforms(),
                              bs=16)

In [ ]:
dls.vocab

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls,resnet50,metrics=accuracy)
learn.fine_tune(10)

In [ ]:
learn = cnn_learner(dls,resnet50,metrics=accuracy,model_dir="/kaggle/working")
learn.lr_find()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=accuracy,model_dir="/kaggle/working")
learn.fit_one_cycle(10, base_lr=1e-3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5, nrows=1, figsize = (25,10))

In [ ]:
test_path = "../input/identify-snake-breed-hackerearth/dataset/test/"
test_image_ids = [i for i in test_df.image_id]
test_images = [test_path+f"{img}.jpg" for img in test_image_ids]
preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))

In [ ]:
preds = preds[0].cpu().numpy()
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
train_df["breed_label"] = lb.fit_transform(train_df.breed)
pred = lb.inverse_transform((np.argmax(preds,axis=1)))

In [ ]:
submission = pd.DataFrame()
submission["image_id"] = test_image_ids
submission['breed'] = pred
submission.to_csv("submission.csv", index=False)

In [ ]:
from fastai.callback.fp16 import *
learn = cnn_learner(dls, resnet50, metrics=accuracy).to_fp16()
learn.fine_tune(8, freeze_epochs=4)

In [ ]:
test_path = "../input/identify-snake-breed-hackerearth/dataset/test/"
test_image_ids = [i for i in test_df.image_id]
test_images = [test_path+f"{img}.jpg" for img in test_image_ids]
preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))

In [ ]:
preds = preds[0].cpu().numpy()
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
train_df["breed_label"] = lb.fit_transform(train_df.breed)
pred = lb.inverse_transform((np.argmax(preds,axis=1)))

In [ ]:
submission = pd.DataFrame()
submission["image_id"] = test_image_ids
submission['breed'] = pred
submission.to_csv("submission.csv", index=False)